# In this notebook I'm generating the movements and the states variables (torques and angles) in order to produce this movement using the 2dof simulator.

### Some of the algorithms (or inspiration) to simulate the 2dof arm came from: http://www.gribblelab.org/compneuro/

# Here starts the 2 joint arm study

# Main functions to the 2 joint arm simulation

In [1]:
# Makes possible to show the output from matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt

# Makes the figures in the PNG format:
# For more information see %config InlineBackend
%config InlineBackend.figure_formats=set([u'png'])

plt.rcParams['figure.figsize'] = 20, 10

import numpy
import sys
import save_load_file as slf

In [4]:
# Loads the modules and starts the object to be used with the parallel processing iPython stuff...

# Remember to execute at the shell: ipcluster start -n 4
# or from the iPython notebook interface!
# from IPython.parallel import Client
from ipyparallel import Client

# When using the ipython in my desktop, launch the cluster in the right profile :)
cli = Client()

# lbview = cli.load_balanced_view()
dview = cli[:]

In [3]:
%%file simulation_2DoF_Arm_physics.py

# 2 DOF Simulator module

import matplotlib.pyplot as plt
import numpy

# forward kinematics
def joints_to_hand(A,aparams):
    """
    Given joint angles A=(a1,a2) and anthropometric params aparams,
    returns hand position H=(hx,hy) and elbow position E=(ex,ey)
    Note1: A must be type matrix (or array([[a1,a2],...]))
    Note2: If A has multiple lines, H and E will have the same number of lines.
    """
    l1 = aparams['l1']
    l2 = aparams['l2']
    n = numpy.shape(A)[0]
    E = numpy.zeros((n,2))
    H = numpy.zeros((n,2))
    for i in range(n):
        E[i,0] = l1 * numpy.cos(A[i,0])
        E[i,1] = l1 * numpy.sin(A[i,0])
        H[i,0] = E[i,0] + (l2 * numpy.cos(A[i,0]+A[i,1]))
        H[i,1] = E[i,1] + (l2 * numpy.sin(A[i,0]+A[i,1]))
    return H,E


# I could do the inverse kinematics using all the possible values (workspace) of the arm and creating a numpy array. Then I 
# could use the argmin trick to find the value.
# In order to solve the problem when multiple solutions appear I could use the minimum jerk criterion. The user should enter
# the actual position and the next one, then the system solves according to the one that uses mininum energy.
# One curious thing about inverse kinematics is that as human beings we cannot do a inverse kinematic of our hand position
# without taking in account the actual position. The function, below, doesn't care about the actual position, and that is why 
# more than one solution appears.
# So, I don't think the brain solves the problem of multiple solutions. Who solves this problem is the morphology of the limbs.
# It is impossible to change trajectories instantaneously, therefore the continuity of the movements is guaranteed.
# Summary: there are no positions, but trajectories :)

# inverse kinematics
def hand_to_joints(H,aparams):
    """
    Given hand position H=(hx,hy) and anthropometric params aparams,
    returns joint angles A=(a1,a2)
    Note1: H must be type matrix (or array([[hx,hy],...]))
    Note2: If H has multiple lines, A will have the same number of lines.
    """
    l1 = aparams['l1']
    l2 = aparams['l2']
    n = numpy.shape(H)[0]
    A = numpy.zeros((n,2))
    for i in range(n):
        A[i,1] = numpy.arccos(((H[i,0]*H[i,0])+(H[i,1]*H[i,1])-(l1*l1)-(l2*l2))/(2.0*l1*l2))
        A[i,0] = numpy.arctan2(H[i,1],H[i,0]) - numpy.arctan2((l2*numpy.sin(A[i,1])),(l1+(l2*numpy.cos(A[i,1]))))
#         if A[i,0] < 0:
#             print "<0:",A[i,0]
#             A[i,0] = A[i,0] + pi
#         elif A[i,0] > pi:
#             print ">0:",A[i,0]
#             A[i,0] = A[i,0] - pi
    return A


# inverse kinematics
def hand_to_joints(H,aparams,ang_error=0.01):
    """
    Given hand position H=(hx,hy) and anthropometric params aparams,
    returns joint angles A=(a1,a2)
    Note1: H must be type matrix (or array([[hx,hy],...]))
    Note2: If H has multiple lines, A will have the same number of lines.
    """
    l1 = aparams['l1']
    l2 = aparams['l2']
    n = numpy.shape(H)[0]
    A = numpy.zeros((n,2))
    t_bias=[0,0] 
    for i in range(n):
        A[i,1] = numpy.arccos(((H[i,0]*H[i,0])+(H[i,1]*H[i,1])-(l1*l1)-(l2*l2))/(2.0*l1*l2)) + t_bias[1]
        A[i,0] = numpy.arctan2(H[i,1],H[i,0]) - numpy.arctan2((l2*numpy.sin(A[i,1])),(l1+(l2*numpy.cos(A[i,1])))) + t_bias[0]
        if i>0:
            # Here I'm trying to avoid descontinuity problems when there's a 2pi difference between them!
            if 0<=abs(abs((A[i,1]-A[i-1,1])/numpy.pi)-2)<=ang_error:
                print "Correction on Joint 2:",(A[i,1],A[i-1,1])
                if (A[i,1]-A[i-1,1])>0:
                    A[i,1]-=2*numpy.pi
                    t_bias[1]-=2*numpy.pi
                else:
                    A[i,1]+=2*numpy.pi
                    t_bias[1]+=2*numpy.pi  
                    
            if 0<=abs(abs((A[i,0]-A[i-1,0])/numpy.pi)-2)<=ang_error:
                print "Correction on Joint 1:",(A[i,0],A[i-1,0])
                if (A[i,0]-A[i-1,0])>0:
                    A[i,0]-=2*numpy.pi
                    t_bias[0]-=2*numpy.pi
                else:
                    A[i,0]+=2*numpy.pi
                    t_bias[0]+=2*numpy.pi
    return A


# Generates the movements according to:
# Flash, Tamar and Neville Hogan. 1985. The Coordination of Arm Movements: An Experimentally Confirmed Mathematical Model. The Journal of Neuroscience 5 (7): 1688-1703
def cartesian_movement_generation_training(xstart,ystart,xdest,ydest,MT,t):
    '''
    xstart,ystart: initial position of the trajectory
    xdest,ydest: final position of the trajectory
    MT: total time spent doing the trajectory
    t: current time
    
    returns a matrix: [[x0,y0],[x1,y1],...]
    '''
    x_t=xstart+(xstart-xdest)*(15*(t/MT)**4-6*(t/MT)**5-10*(t/MT)**3)
    y_t=ystart+(ystart-ydest)*(15*(t/MT)**4-6*(t/MT)**5-10*(t/MT)**3)    
    return numpy.array([x_t,y_t]).T


# Used to generate the velocities and the accelerations using the position and time vectors
def derivator(v,t):
    return numpy.array([(v[i+1]-v[i])/(t[i+1]-t[i]) for i in range(len(t)-1)])


def twojointarm_torques(state, t, aparams):
    """
    Calculates the necessaries torques to generate the accelerations
    """
    import numpy

    a1,a2,a1d,a2d,a1dd,a2dd = state # joint_angle_a1,joint_angle_a2,joint_vel_a1,joint_vel_a2,joint_acc_a1,joint_acc_a2

    l1,l2 = aparams['l1'], aparams['l2'] # lenght link 1 and 2
    m1,m2 = aparams['m1'], aparams['m2'] # mass link 1 and 2
    i1,i2 = aparams['i1'], aparams['i2'] # moment of inertia link 1 and 2
    lc1,lc2 = aparams['lc1'], aparams['lc2'] # distance to the center of mass of link 1 and 2

    M11 = i1 + i2 + (m1*lc1*lc1) + (m2*((l1*l1) + (lc2*lc2) + (2*l1*lc2*numpy.cos(a2))))
    M12 = i2 + (m2*((lc2*lc2) + (l1*lc2*numpy.cos(a2))))
    M21 = M12
    M22 = i2 + (m2*lc2*lc2)
    M = numpy.matrix([[M11,M12],[M21,M22]]) # H matrix

    C1 = -(m2*l1*a2d*a2d*lc2*numpy.sin(a2)) - (2*m2*l1*a1d*a2d*lc2*numpy.sin(a2))
    C2 = m2*l1*a1d*a1d*lc2*numpy.sin(a2)
    C = numpy.matrix([[C1],[C2]])

    ACC = numpy.array([[a1dd],[a2dd]])

    T = M*ACC + C

    return numpy.array([T[0,0],T[1,0]])


# forward dynamics equations of our two-joint arm
def twojointarm(state, t, aparams, torque):
    import numpy
    
    """
    two-joint arm in plane
    X is fwd(+) and back(-)
    Y is up(+) and down(-)
    shoulder angle a1 relative to Y vert, +ve counter-clockwise
    elbow angle a2 relative to upper arm, +ve counter-clockwise
    """
    a1,a2,a1d,a2d = state # joint_angle_a1, joint_angle_a2, joint_velocity_a1, joint_velocity_a2

    l1,l2 = aparams['l1'], aparams['l2'] # lenght link 1 and 2
    m1,m2 = aparams['m1'], aparams['m2'] # mass link 1 and 2
    i1,i2 = aparams['i1'], aparams['i2'] # moment of inertia link 1 and 2
    lc1,lc2 = aparams['lc1'], aparams['lc2'] # distance to the center of mass of link 1 and 2

    M11 = i1 + i2 + (m1*lc1*lc1) + (m2*((l1*l1) + (lc2*lc2) + (2*l1*lc2*numpy.cos(a2))))
    M12 = i2 + (m2*((lc2*lc2) + (l1*lc2*numpy.cos(a2))))
    M21 = M12
    M22 = i2 + (m2*lc2*lc2)
    M = numpy.matrix([[M11,M12],[M21,M22]]) # H matrix

    C1 = -(m2*l1*a2d*a2d*lc2*numpy.sin(a2)) - (2*m2*l1*a1d*a2d*lc2*numpy.sin(a2))
    C2 = m2*l1*a1d*a1d*lc2*numpy.sin(a2)
    C = numpy.matrix([[C1],[C2]])

    T = numpy.matrix([[torque[0]],[torque[1]]])

    ACC = numpy.linalg.inv(M) * (T-C) # calculates the accelerations of joints 1 and 2

    a1dd,a2dd = ACC[0,0], ACC[1,0]

    return [a1d, a2d, a1dd, a2dd] # It returns the first and second derivatives of the joints


def animatearm(state,t,aparams):
    """
    animate the twojointarm
    """
    import matplotlib.pyplot as plt
    import numpy
    import time

    A = state[:,[0,1]] # Gets the angles a1 and a2 from the states matrix
    A[:,0] = A[:,0]
    H,E = joints_to_hand(A,aparams)
    l1,l2 = aparams['l1'], aparams['l2']
    plt.figure()
    plt.plot(0,0,'b.')
    plt.plot(H[:,0],H[:,1],'g.-');
    p1, = plt.plot(E[0,0],E[0,1],'b.')
    p2, = plt.plot(H[0,0],H[0,1],'b.')
    p3, = plt.plot((0,E[0,0],H[0,0]),(0,E[0,1],H[0,1]),'b-')
    plt.xlim([-l1-l2, l1+l2])
    plt.ylim([-l1-l2, l1+l2])
    dt = t[1]-t[0]
    tt = plt.title("Click on this plot to continue...")
    plt.ginput(1)
    for i in xrange(0,numpy.shape(state)[0]):
        time.sleep(0.05)
        p1.set_xdata((E[i,0]))
        p1.set_ydata((E[i,1]))
        p2.set_xdata((H[i,0]))
        p2.set_ydata((H[i,1]))
        p3.set_xdata((0,E[i,0],H[i,0]))
        p3.set_ydata((0,E[i,1],H[i,1]))
        tt.set_text("Current time:%4.2f sec - click to next slide!" % (i*dt))
        plt.draw()
    tt.set_text("Current time:%4.2f sec - finished!" % ((numpy.shape(state)[0]-1)*dt))
    plt.draw()        

    
def animatearm_JS(state,t,aparams):
    """
    animate the twojointarm
    """
    import matplotlib.pyplot as plt
    import numpy
    from JSAnimation import IPython_display
    from matplotlib import animation


    A = state[:,[0,1]] # Gets the angles a1 and a2 from the states matrix
    A[:,0] = A[:,0]
    H,E = joints_to_hand(A,aparams)
    l1,l2 = aparams['l1'], aparams['l2']

    # Set up the axes, making sure the axis ratio is equal
#     ax = fig.add_axes([0, 0, 1, 1], xlim=(-0.02, 13.02), ylim=(-0.02, 5.02),
#                       xticks=range(14), yticks=range(6), aspect='equal', frameon=False)

    fig = plt.figure(figsize=(6, 6),dpi=100)
    ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1), aspect='equal')
    ax.plot(0,0,'b.')
    ax.plot(H[:,0],H[:,1],'g.-');
    p1, = ax.plot(E[0,0],E[0,1],'b.')
    p2, = ax.plot(H[0,0],H[0,1],'b.')
    p3, = ax.plot((0,E[0,0],H[0,0]),(0,E[0,1],H[0,1]),'b-')    
    
    def init():
        p1.set_data([],[])
        p2.set_data([],[])
        p3.set_data([],[])        
        return p1,p2,p3
    
    def animate(i):
        p1.set_data([E[i,0]],[E[i,1]])
        p2.set_data(H[i,0],H[i,1])
        p3.set_data((0,E[i,0],H[i,0]),(0,E[i,1],H[i,1]))
        return p1,p2,p3
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(E[:,0]), interval=20, blit=True)
    return anim
    # In order to make the JSAnimation to work it is necessary that the function returns the animation object!
    
    
def odeint_arms(twojointarm, state, t, aparams, torque):
    '''
    twojointarm: function object. Must receive (state,t,aparams,torque) and return [a1d,a2d,a1dd,a2dd]
    state: current states => [a1(t),a2(t),a1d(t),a2d(t)]
    t: array([t,t+1]) => current time step and next (t+1)
    returns next states [a1(t+1),a2(t+1),a1d(t+1),a2d(t+1)]
    '''
    from scipy.integrate import odeint

    return odeint(twojointarm, state, t, args=(aparams,torque))


def moving_average (values,window=6):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(numpy.concatenate((numpy.zeros(int((window-1)/2.0)),values,numpy.zeros((window-1)-int((window-1)/2.0)))), weights, 'valid')
    # I should try the function numpy.lib.pad instead of concatenating manually
    return sma


def moving_average (values, window=6):
    weights = numpy.repeat(1.0, window)/window
    sma = numpy.convolve(values, weights, 'valid')
    # I should try the function numpy.lib.pad instead of concatenating manually
    return numpy.lib.pad(sma, (int((window-1)/2.0),(window-1)-int((window-1)/2.0)), 'edge')

Overwriting simulation_2DoF_Arm_physics.py


In [4]:
from simulation_2DoF_Arm_physics import *
reload(sys.modules['simulation_2DoF_Arm_physics']) # Makes sure the interpreter is going to reload the module

<module 'simulation_2DoF_Arm_physics' from 'simulation_2DoF_Arm_physics.pyc'>

In [5]:
@dview.parallel(block=True)
def generate_trajectories(sim_inputs):
    import numpy
    import sys
    import save_load_file as slf
    
    import simulation_2DoF_Arm_physics
    reload(sys.modules['simulation_2DoF_Arm_physics']) # Makes sure the interpreter is going to reload the module
    s2ap = simulation_2DoF_Arm_physics

    tji,positions,sim_params = sim_inputs
    
    xstart,ystart = positions[0]
    xdest,ydest = positions[1]

    sim_set,base_dir,MT,time_step,Ninput,aparams = sim_params
    
    t_mov=numpy.arange(0, MT, time_step) # t starts in 0s and steps time_step(s) until reaches MT(s)

    # Generates the movements according to:
    # Flash, Tamar, and Neville Hogan. 1985
    H_path=s2ap.cartesian_movement_generation_training(xstart, ystart, xdest, ydest, MT, t_mov)

    # These are the values teta1 and teta2 can have because the system limits the resolution.
    # According to Joshi/Maass paper there are 50 neurons to code the positions of each variable.
    #
    #
    teta1=numpy.linspace(-numpy.pi/6,numpy.pi,num=Ninput)
    teta2=numpy.linspace(0,numpy.pi,num=Ninput)
    teta1_teta2 = numpy.array([teta1,teta2]).T # This is the matrix to use with the function
                                               # to generate the x,y values of the workspace

    # Joint's workspace: all the possible combinations between teta1 and teta2.
    teta_workspace = numpy.array([[t1,t2] for t1 in teta1 for t2 in teta2])

    # Arm's workspace: x,y points that the arm (endpoint) can reach
    H_workspace = s2ap.joints_to_hand(teta_workspace,aparams)[0] # I'm getting the first because it returns the elbow's positions too.

    # Generate the joint's positions according to the ORIGINAL (X,Y) values.
    # I'm using the traditional geometric way to do the inverse kinematics here. I need to
    # implement the minimum jerk way to generate the joint's positions taking into account the movement's dynamics.

    Joints=s2ap.hand_to_joints(H_path, aparams,ang_error=0.1)

    # Here I'm extending the Joints matrix because I need two extra positions to calculate the accelerations.
    # Consequently, because the trajectory always finish with velocity ZERO, keeping the same position seems a good choice. 
    Joints_extended=numpy.concatenate((Joints,[Joints[-1],Joints[-1]]))

    # But the time array (t_mov) must be extended too:
    t_mov_extended=numpy.concatenate((t_mov,[t_mov[-1]+time_step],[t_mov[-1]+2*time_step]))


    # Joint's velocities
    teta1_d=s2ap.derivator(Joints_extended[:,0],t_mov_extended)
    teta2_d=s2ap.derivator(Joints_extended[:,1],t_mov_extended)
    

    # Joint's accelerations
    teta1_dd=s2ap.derivator(teta1_d,t_mov_extended[:251])
    teta2_dd=s2ap.derivator(teta2_d,t_mov_extended[:251])

    #
    # WITH ORIGINAL JOINT'S VELOCITIES
    #

    # And generates a matrix with [teta1,teta2,teta1d,teta2d,teta1dd,teta2dd]
    # Using this matrix I will generate the torques.
    states_mov = numpy.array([[Joints[:,0][i],Joints[:,1][i],teta1_d[i],teta2_d[i],teta1_dd[i],teta2_dd[i]] for i in range(len(Joints[:,0]))])


    # Applying the function to all lines of the states_mov matrix, I generate the torques matrix
    T_mov=numpy.array([s2ap.twojointarm_torques(states_mov[i], t_mov, aparams) for i in range(numpy.shape(states_mov)[0])])

    # Here I calculate the states using the calculated torques, just to make sure it is working!
    state = states_mov[0,[0,1,2,3]] # This is the initial state. The rest of the states will be generated dynamically 
                                    # according to the input torques.
    state_v2 = [state]
    t = t_mov
    torque = T_mov
    for i in range(len(t)-1):
        print "state:",state
        state = s2ap.odeint_arms(s2ap.twojointarm, state, [t[i], t[i+1]], aparams, torque[i])[1]
        state_v2.append(state)

    state_v2=numpy.array(state_v2)

    import save_load_file as slf
    slf.save_to_file_gz([numpy.array(state_v2),numpy.array(torque)],"./"+base_dir+"/"+sim_set+"/States_Torques_movement"+str(tji)+".gzpickle")
    
    return tji

## End of the main functions!

# Adjusting the parameters:

In [6]:
# Experiment identifier
sim_set = "set_09"

# Base dir to save / access
base_dir = "2DofArm_simulation_data"

# List with all trajectories to be generated
# [[[start_x,start_y],[final_x,final_y]],...]
trajectories = [[[0.75,0.25],[0.0,0.5]], [[0.25,0.60],[-0.25,0.60]], [[-0.10,0.75],[-0.10,0.25]],[[-0.75,0.50],[-0.40,0.00]]]


# The values below must match the ones used with the SNN simulation:

# Total time spent during the movement (in seconds)
MT = 0.5

# Simulation time step (in seconds)
time_step = 2/1000.0

# Number of neurons at the input layer (defines the resolution of the system)
Ninput = 50


# Arm parametres used with the 2 dof arm simulator
# (according to Joshi/Maass 2006 paper)
aparams = {
    'l1' : 0.5, # metres
    'l2' : 0.5,
    'lc1' : 0.25,
    'lc2' : 0.25,
    'm1' : 1.0, # kg
    'm2' : 1.0,
    'i1' : 0.03, # kg*m*m
    'i2' : 0.03
}

# Variable only used to pass the parameters
sim_params = sim_set,base_dir,MT,time_step,Ninput,aparams

In [7]:
%time results = generate_trajectories.map([(tji,positions,sim_params) for tji,positions in zip(range(1,len(trajectories)+1),trajectories)])

CPU times: user 68.3 ms, sys: 5.84 ms, total: 74.1 ms
Wall time: 718 ms


#STOP HERE!
#THE DATA GENERATION STOPS AT THE ABOVE CELL.

# Study to identify outliers when using random trajectories

### Because the system used in Joshi/Maass 2006 accepts only 50 possible values for the input variables, becomes impossible to generate trajectories that need very different ranges of values. The multilayer encoding system I developed could improve this, because with 50 neurons I could generate about 100,000 values instead of only 50.

In [101]:
#
# Here I want to identify the outliers. Some trajectories are going to generate torque 
# values that are unfeasible (almost Dirac's deltas). I need to check which ones are outliers
# and generate a list of their index so I can ignore them during the linear regression.
#


# Experiment identifier
sim_set = "set_03n"

# Base dir to save / access
base_dir = "2DofArm_simulation_data_new"

number_of_trajectories = 40

# These values are based ONLY in the "number_of_trajectories" trajectories used!
torques_all_1 = numpy.array([]);
torques_all_2 = numpy.array([]);
for i in xrange(1,number_of_trajectories+1):
    states,torques = slf.load_from_file_gz("./"+base_dir+"/"+sim_set+"/States_Torques_movement"+str(i)+".gzpickle")
    torques_all_1 = numpy.concatenate((torques_all_1,torques[:,0])) # torques joint 1
    torques_all_2 = numpy.concatenate((torques_all_2,torques[:,1])) # torques joint 2

abs_tau1 = 2*torques_all_1.std()
abs_tau2 = 2*torques_all_2.std()


# Now I'm going to run through the files again (it's quite fast) to check
outliers_list = []
for i in xrange(1,number_of_trajectories+1):
    states,torques = slf.load_from_file_gz("./"+base_dir+"/"+sim_set+"/States_Torques_movement"+str(i)+".gzpickle")
    if (abs(torques[:,0])).max()>abs_tau1 or (abs(torques[:,1])).max()>abs_tau2:
        outliers_list.append(i)

torques_all_1_o = numpy.array([]);
torques_all_2_o = numpy.array([]);
for i in xrange(1,number_of_trajectories+1):
    if i not in outliers_list:
        states,torques = slf.load_from_file_gz("./"+base_dir+"/"+sim_set+"/States_Torques_movement"+str(i)+".gzpickle")
        torques_all_1_o = numpy.concatenate((torques_all_1_o,torques[:,0])) # torques joint 1
        torques_all_2_o = numpy.concatenate((torques_all_2_o,torques[:,1])) # torques joint 2
        

# Creates the 50 possible discrete values of torques for each joint (probably they will be different)
torques_values_1=numpy.linspace(torques_all_1.min(),torques_all_1.max(),num=50)
torques_values_2=numpy.linspace(torques_all_2.min(),torques_all_2.max(),num=50)


In [102]:
(torques_all_1.max(),torques_all_1.min()),(torques_all_2.max(),torques_all_2.min()),torques_all_1.std(),torques_all_2.std(),abs(torques_all_1).mean(),abs(torques_all_2).mean()

((4724.3458910714662, -4410.3172561201773),
 (794.19786205423236, -725.15082517018777),
 137.32887607297502,
 23.944553955114543,
 17.129325790345458,
 3.5209663649915912)

In [103]:
(torques_all_1_o.max(),torques_all_1_o.min()),\
(torques_all_2_o.max(),torques_all_2_o.min()),\
torques_all_1_o.std(),torques_all_2_o.std(),\
abs(torques_all_1_o).mean(),abs(torques_all_2_o).mean()


((129.8487789837329, -121.48152348028766),
 (23.380258628971585, -23.210738884167242),
 14.900957324003183,
 2.9264069203116243,
 8.5852235920490756,
 1.9168447675100866)